In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

In [2]:
df = pd.read_csv('resources/fifa-18-demo-player-dataset/CompleteDataset.csv')
df.drop(['Unnamed: 0', 'Flag', 'Club Logo', 'Value', 'Wage', 'Photo'], axis=1, inplace=True)
df[df['Club']=='Fiorentina']

/Users/Michael/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Name,Age,Nationality,Overall,Potential,Club,Special,Acceleration,Aggression,Agility,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
373,D. Astori,30,Italy,81,81,Fiorentina,1656,66,77,57,...,73.0,80.0,62.0,75.0,54.0,56.0,55.0,53.0,70.0,55.0
544,M. Benassi,22,Italy,79,86,Fiorentina,2086,69,74,75,...,74.0,74.0,78.0,76.0,75.0,75.0,74.0,75.0,75.0,74.0
583,R. Saponara,25,Italy,79,82,Fiorentina,1864,71,52,71,...,55.0,47.0,74.0,58.0,77.0,78.0,72.0,77.0,59.0,72.0
722,M. Sportiello,25,Italy,78,82,Fiorentina,1234,48,23,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
770,V. Eysseric,25,France,78,80,Fiorentina,1941,72,66,76,...,55.0,49.0,73.0,59.0,77.0,77.0,74.0,78.0,60.0,74.0
774,M. Badelj,28,Croatia,78,78,Fiorentina,2012,58,76,67,...,74.0,74.0,77.0,78.0,70.0,72.0,67.0,70.0,75.0,67.0
831,V. Eysseric,25,France,78,80,Fiorentina,1941,72,66,76,...,55.0,49.0,73.0,59.0,77.0,77.0,74.0,78.0,60.0,74.0
864,M. Badelj,28,Croatia,78,78,Fiorentina,2012,58,76,67,...,74.0,74.0,77.0,78.0,70.0,72.0,67.0,70.0,75.0,67.0
973,Vitor Hugo,26,Brazil,77,79,Fiorentina,1563,66,80,43,...,67.0,76.0,46.0,63.0,44.0,45.0,48.0,43.0,63.0,48.0
975,J. Veretout,24,France,77,83,Fiorentina,2025,68,65,69,...,70.0,67.0,76.0,73.0,74.0,75.0,71.0,74.0,72.0,71.0


In [3]:
positions_list = list(df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).drop_duplicates())
positions_list

['ST',
 'RW',
 'LW',
 'GK',
 'CDM',
 'CB',
 'RM',
 'CM',
 'LM',
 'LB',
 'CAM',
 'RB',
 'CF',
 'RWB',
 'LWB']

In [4]:
positions_dict = {'ST': 'Attack',
                  'RW': 'Attack',
                  'LW': 'Attack',
                  'GK': 'Defense',
                  'CDM': 'Midfield',
                  'CB': 'Defense',
                  'RM': 'Midfield',
                  'CM': 'Midfield',
                  'LM': 'Midfield',
                  'LB': 'Defense',
                  'CAM': 'Midfield',
                  'RB': 'Defense',
                  'CF': 'Attack',
                  'RWB': 'Defense',
                  'LWB': 'Defense'}

In [5]:
df2 = df[['Name', 'Club', 'Nationality', 'Preferred Positions', 'Overall']].copy()
df2['Preferred Positions'] = df2['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).replace(positions_dict)

team = 'Brazil'
df2[df2['Nationality'] == team].head(14).groupby(['Nationality', 'Preferred Positions']).mean()

Overall
Nationality Preferred Positions         
Brazil      Attack                 92.00
            Defense                85.25
            Midfield               84.80

In [6]:
epl2018 = pd.read_csv('resources/leagues/epl2018.csv')
epl2018.head()

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,0,E0,11/08/17,Arsenal,Leicester,4,3,H,2,2,...,2.32,21,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25
1,1,E0,12/08/17,Brighton,Man City,0,2,A,0,0,...,2.27,20,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29
2,2,E0,12/08/17,Chelsea,Burnley,2,3,A,0,3,...,2.23,20,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25
3,3,E0,12/08/17,Crystal Palace,Huddersfield,0,3,A,0,2,...,1.72,18,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51
4,4,E0,12/08/17,Everton,Stoke,1,0,H,1,0,...,1.76,19,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42


In [7]:
bundesliga2018 = pd.read_csv('resources/leagues/bundesliga2018.csv')
bundesliga2018.head()

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,0,D1,18/08/17,Bayern Munich,Leverkusen,3,1,H,2,0,...,2.88,17,-2.00,2.26,2.18,1.77,1.72,1.28,6.70,11.30
1,1,D1,19/08/17,Hamburg,Augsburg,1,0,H,1,0,...,1.68,18,-0.25,1.88,1.84,2.07,2.03,2.32,3.32,3.47
2,2,D1,19/08/17,Hertha,Stuttgart,2,0,H,0,0,...,1.87,18,-0.50,2.09,2.05,1.87,1.83,2.02,3.60,4.07
3,3,D1,19/08/17,Hoffenheim,Werder Bremen,1,0,H,0,0,...,2.50,16,-0.75,2.04,1.98,1.91,1.88,1.85,4.19,4.12
4,4,D1,19/08/17,Mainz,Hannover,0,1,A,0,0,...,1.76,16,-0.50,2.01,1.97,1.93,1.89,1.85,3.75,4.65


In [8]:
seriea2018 = pd.read_csv('resources/leagues/seriea2018.csv')
seriea2018.head()

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,0,I1,19/08/17,Juventus,Cagliari,3,0,H,2,0,...,2.41,16,-2.00,1.83,1.80,2.13,2.07,1.13,9.74,27.70
1,1,I1,19/08/17,Verona,Napoli,1,3,A,0,2,...,2.29,16,1.25,1.98,1.95,1.97,1.92,8.37,5.21,1.41
2,2,I1,20/08/17,Atalanta,Roma,0,1,A,0,1,...,2.06,16,0.25,1.91,1.88,2.04,1.99,2.99,3.44,2.53
3,3,I1,20/08/17,Bologna,Torino,1,1,D,1,1,...,1.75,16,0.25,1.90,1.87,2.06,2.00,3.35,3.47,2.31
4,4,I1,20/08/17,Crotone,Milan,0,3,A,0,3,...,2.06,17,1.00,1.89,1.85,2.08,2.02,7.42,4.37,1.52


In [9]:
laliga2018 = pd.read_csv('resources/leagues/laliga2018.csv')
laliga2018.head()

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,0,SP1,18/08/17,Leganes,Alaves,1,0,H,1,0,...,1.46,18,-0.50,2.07,2.03,1.90,1.86,1.98,3.35,4.63
1,1,SP1,18/08/17,Valencia,Las Palmas,1,0,H,1,0,...,2.27,16,-0.75,2.05,1.97,1.96,1.91,1.78,4.24,4.43
2,2,SP1,19/08/17,Celta,Sociedad,2,3,A,1,1,...,1.84,18,-0.25,2.08,2.05,1.87,1.83,2.12,3.53,3.74
3,3,SP1,19/08/17,Girona,Ath Madrid,2,2,D,2,0,...,1.74,16,1.25,1.77,1.75,2.25,2.16,6.93,3.83,1.63
4,4,SP1,19/08/17,Sevilla,Espanol,1,1,D,1,1,...,2.09,16,-1.00,2.12,2.06,1.86,1.82,1.64,4.18,5.82


In [10]:
results_dict = {'H': 2, "D": 1, "A": 0}
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
test_df = epl2018[cols].rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals', 'FTR': 'Results'}).copy()
test_df['Results'] = test_df['Results'].replace(results_dict)
test_df

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Results
0,Arsenal,Leicester,4,3,2
1,Brighton,Man City,0,2,0
2,Chelsea,Burnley,2,3,0
3,Crystal Palace,Huddersfield,0,3,0
4,Everton,Stoke,1,0,2
5,Southampton,Swansea,0,0,1
6,Watford,Liverpool,3,3,1
7,West Brom,Bournemouth,1,0,2
8,Man United,West Ham,4,0,2
9,Newcastle,Tottenham,0,2,0


In [11]:
player_overall_df = df[['Name', 'Club', 'Nationality', 'Preferred Positions', 'Overall']].copy()
player_overall_df['Preferred Positions'] = player_overall_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).replace(positions_dict)

In [12]:
def get_team_stats(team, type='Club', no_of_players=11, return_df=False):
    mask = df2[type] == team
    groupby_cols = [type, 'Preferred Positions']
    team_stats_df = player_overall_df[mask].head(no_of_players).groupby(groupby_cols).mean()
    if return_df == True:
        return team_stats_df
    else:
        df = team_stats_df.reset_index().pivot(type, 'Preferred Positions')
        return df['Overall'].values.tolist()

In [13]:
get_team_stats('Chelsea', return_df=True)

Overall
Club    Preferred Positions           
Chelsea Attack               86.000000
        Defense              86.000000
        Midfield             85.666667

In [14]:
get_team_stats('Chelsea')

[[86.0, 86.0, 85.66666666666667]]

In [15]:
epl_teams = list(test_df['HomeTeam'].drop_duplicates().values)
epl_teams

['Arsenal',
 'Brighton',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Southampton',
 'Watford',
 'West Brom',
 'Man United',
 'Newcastle',
 'Bournemouth',
 'Burnley',
 'Leicester',
 'Liverpool',
 'Stoke',
 'Swansea',
 'Huddersfield',
 'Tottenham',
 'Man City',
 'West Ham']

In [16]:
epl_team_dict = {'Arsenal': 'Arsenal',
                 'Brighton': 'Brighton & Hove Albion',
                 'Chelsea': 'Chelsea',
                 'Crystal Palace': 'Crystal Palace',
                 'Everton': 'Everton',
                 'Southampton': 'Southampton',
                 'Watford': 'Watford',
                 'West Brom': 'West Bromwich Albion',
                 'Man United': 'Manchester United',
                 'Newcastle': 'Newcastle United',
                 'Bournemouth': 'Bournemouth',
                 'Burnley': 'Burnley',
                 'Leicester': 'Leicester City',
                 'Liverpool': 'Liverpool',
                 'Stoke': 'Stoke City',
                 'Swansea': 'Swansea City',
                 'Huddersfield': 'Huddersfield Town',
                 'Tottenham': 'Tottenham Hotspur',
                 'Man City': 'Manchester City',
                 'West Ham': 'West Ham United'}

In [17]:
bund_teams = list(bundesliga2018['HomeTeam'].drop_duplicates().values)
bund_teams

['Bayern Munich',
 'Hamburg',
 'Hertha',
 'Hoffenheim',
 'Mainz',
 'Schalke 04',
 'Wolfsburg',
 'Freiburg',
 "M'gladbach",
 'FC Koln',
 'Augsburg',
 'Dortmund',
 'Ein Frankfurt',
 'Leverkusen',
 'Stuttgart',
 'Werder Bremen',
 'Hannover',
 'RB Leipzig']

In [18]:
bundesliga_team_dict = {'Bayern Munich': 'FC Bayern Munich',
                        'Hamburg': 'Hamburger SV',
                        'Hertha': 'Hertha BSC Berlin',
                        'Hoffenheim': 'TSG 1899 Hoffenheim',
                        'Mainz': '1. FSV Mainz 05',
                        'Schalke 04': 'FC Schalke 04',
                        'Wolfsburg': 'VfL Wolfsburg',
                        'Freiburg': 'SC Freiburg',
                        "M'gladbach": 'Borussia Mönchengladbach',
                        'FC Koln': '1. FC Köln',
                        'Augsburg': 'FC Augsburg',
                        'Dortmund': 'Borussia Dortmund',
                        'Ein Frankfurt': 'Eintracht Frankfurt',
                        'Leverkusen': 'Bayer 04 Leverkusen',
                        'Stuttgart': 'VfB Stuttgart',
                        'Werder Bremen': 'SV Werder Bremen',
                        'Hannover': 'Hannover 96',
                        'RB Leipzig': 'RB Leipzig'}

In [19]:
laliga_teams = list(laliga2018['HomeTeam'].drop_duplicates().values)
laliga_teams

['Leganes',
 'Valencia',
 'Celta',
 'Girona',
 'Sevilla',
 'Ath Bilbao',
 'Barcelona',
 'La Coruna',
 'Levante',
 'Malaga',
 'Betis',
 'Sociedad',
 'Alaves',
 'Las Palmas',
 'Eibar',
 'Espanol',
 'Getafe',
 'Real Madrid',
 'Villarreal',
 'Ath Madrid']

In [20]:
laliga_team_dict = {'Leganes': 'CD Leganés',
                    'Valencia': 'Valencia CF',
                    'Celta': 'RC Celta de Vigo',
                    'Girona': 'Girona CF',
                    'Sevilla': 'Sevilla FC',
                    'Ath Bilbao': 'Athletic Club de Bilbao',
                    'Barcelona': 'FC Barcelona',
                    'La Coruna': 'RC Deportivo de La Coruña',
                    'Levante': 'Levante UD',
                    'Malaga': 'Málaga CF',
                    'Betis': 'Real Betis Balompié',
                    'Sociedad': 'Real Sociedad',
                    'Alaves': 'Deportivo Alavés',
                    'Las Palmas': 'UD Las Palmas',
                    'Eibar': 'SD Eibar',
                    'Espanol': 'RCD Espanyol',
                    'Getafe': 'Getafe CF',
                    'Real Madrid': 'Real Madrid CF',
                    'Villarreal': 'Villarreal CF',
                    'Ath Madrid': 'Atlético Madrid'
                    }

In [21]:
seriea_teams = list(seriea2018['HomeTeam'].drop_duplicates().values)
seriea_teams

['Juventus',
 'Verona',
 'Atalanta',
 'Bologna',
 'Crotone',
 'Inter',
 'Lazio',
 'Sampdoria',
 'Sassuolo',
 'Udinese',
 'Benevento',
 'Genoa',
 'Roma',
 'Chievo',
 'Fiorentina',
 'Milan',
 'Napoli',
 'Spal',
 'Torino',
 'Cagliari']

In [22]:
seriea_teams_dict = {'Juventus': 'Juventus',
                     'Verona': 'Hellas Verona',
                     'Atalanta': 'Atalanta',
                     'Bologna': 'Bologna',
                     'Crotone': 'Crotone',
                     'Inter': 'Inter',
                     'Lazio': 'Lazio',
                     'Sampdoria': 'Sampdoria',
                     'Sassuolo': 'Sassuolo',
                     'Udinese': 'Udinese',
                     'Benevento': 'Benevento Calcio',
                     'Genoa': 'Genoa',
                     'Roma': 'Roma',
                     'Chievo': 'Chievo Verona',
                     'Fiorentina': 'Fiorentina',
                     'Milan': 'Milan',
                     'Napoli': 'Napoli',
                     'Spal': 'Ferrara (SPAL)',
                     'Torino': 'Torino',
                     'Cagliari': 'Cagliari'
                    }

In [23]:
team_cols = ['HomeTeam', 'AwayTeam']
epl2018[team_cols] = epl2018[team_cols].replace(epl_team_dict)
bundesliga2018[team_cols] = bundesliga2018[team_cols].replace(bundesliga_team_dict)
seriea2018[team_cols] = seriea2018[team_cols].replace(seriea_teams_dict)
laliga2018[team_cols] = laliga2018[team_cols].replace(laliga_team_dict)

In [24]:
seriea2018['HomeTeam'].drop_duplicates().sort_values()

2             Atalanta
10    Benevento Calcio
3              Bologna
24            Cagliari
13       Chievo Verona
4              Crotone
18      Ferrara (SPAL)
15          Fiorentina
11               Genoa
1        Hellas Verona
5                Inter
0             Juventus
6                Lazio
16               Milan
17              Napoli
12                Roma
7            Sampdoria
8             Sassuolo
19              Torino
9              Udinese
Name: HomeTeam, dtype: object

In [25]:
results_dict = {'H': 'HomeWin', "D": 'Draw', "A": 'AwayWin'}
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

league_results_df = pd.concat([epl2018, bundesliga2018, seriea2018, laliga2018])[cols]
league_results_df.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals', 'FTR': 'Result'}, inplace=True)
league_results_df['Result'] = league_results_df['Result'].replace(results_dict)
league_results_df['Goal diff'] = league_results_df['HomeGoals'] - league_results_df['AwayGoals']

league_results_df.reset_index(drop=True, inplace=True)
league_results_df

/Users/Michael/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.path.


,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result,Goal diff
0,Arsenal,Leicester City,4,3,HomeWin,1
1,Brighton & Hove Albion,Manchester City,0,2,AwayWin,-2
2,Chelsea,Burnley,2,3,AwayWin,-1
3,Crystal Palace,Huddersfield Town,0,3,AwayWin,-3
4,Everton,Stoke City,1,0,HomeWin,1
5,Southampton,Swansea City,0,0,Draw,0
6,Watford,Liverpool,3,3,Draw,0
7,West Bromwich Albion,Bournemouth,1,0,HomeWin,1
8,Manchester United,West Ham United,4,0,HomeWin,4
9,Newcastle United,Tottenham Hotspur,0,2,AwayWin,-2


In [26]:
# teams_list = list(league_results_df['HomeTeam'].drop_duplicates())
# team_rating_list = []

# teams_list[0]

# get_team_stats(teams_list[0])

# for team in teams_list:
#     print(get_team_stats(team))

# for team in teams_list:
#     rating = get_team_stats(team)
#     team_rating_list.append(rating)
    
# team_ratings = pd.DataFrame(data=team_rating_list, index=teams_list, columns=['team_rating']).sort_index()
# team_ratings_dict = team_ratings.to_dict()
# team_ratings_dict

In [27]:
# team_cols = ['HomeTeam', 'AwayTeam']
# league_results_df[team_cols] = league_results_df[team_cols].replace(team_ratings_dict['team_rating'])
# league_results_df

In [28]:
league_results_df.tail()

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result,Goal diff
1441,Villarreal CF,Real Madrid CF,2,2,Draw,0
1442,Athletic Club de Bilbao,RCD Espanyol,0,1,AwayWin,-1
1443,Atlético Madrid,SD Eibar,2,2,Draw,0
1444,FC Barcelona,Real Sociedad,1,0,HomeWin,1
1445,Valencia CF,RC Deportivo de La Coruña,2,1,HomeWin,1


In [29]:
homeTeam_attack = []
homeTeam_mid = []
homeTeam_def = []
awayTeam_attack = []
awayTeam_mid = []
awayTeam_def = []
data = []


for index, row in league_results_df.iterrows():
    try:
        home_team_stats = get_team_stats(row['HomeTeam'])
        home_attack = home_team_stats[0][0]
        home_midfield = home_team_stats[0][1]
        home_defense = home_team_stats[0][2]

        away_team_stats = get_team_stats(row['AwayTeam'])
        away_attack = away_team_stats[0][0]
        away_midfield = away_team_stats[0][1]
        away_defense = away_team_stats[0][2]
        
        if len(home_team_stats[0]) != 3 or len(away_team_stats[0]) != 3:
            homeTeam_attack.append(None)
            homeTeam_mid.append(None)
            homeTeam_def.append(None)
            awayTeam_attack.append(None)
            awayTeam_mid.append(None)
            awayTeam_def.append(None)
        else:            
            homeTeam_attack.append(home_attack)
            homeTeam_mid.append(home_midfield)
            homeTeam_def.append(home_defense)
            awayTeam_attack.append(away_attack)
            awayTeam_mid.append(away_midfield)
            awayTeam_def.append(away_defense)

        
    except IndexError:
        homeTeam_attack.append(None)
        homeTeam_mid.append(None)
        homeTeam_def.append(None)
        awayTeam_attack.append(None)
        awayTeam_mid.append(None)
        awayTeam_def.append(None)
        continue

In [30]:
data = [homeTeam_attack, 
        homeTeam_mid, 
        homeTeam_def, 
        awayTeam_attack,
        awayTeam_mid, 
        awayTeam_def]

feature_cols = ['homeTeam_attack', 
                'homeTeam_mid', 
                'homeTeam_def', 
                'awayTeam_attack', 
                'awayTeam_mid',
                'awayTeam_def']

team_ratings_df = pd.DataFrame(data, index=feature_cols)

In [31]:
team_ratings_df = team_ratings_df.transpose()
team_ratings_df['result'] = league_results_df['Result']

In [32]:
team_ratings_df.dropna(inplace=True)

In [33]:
X = team_ratings_df[feature_cols]
y = team_ratings_df['result']

In [34]:
X.head()

,homeTeam_attack,homeTeam_mid,homeTeam_def,awayTeam_attack,awayTeam_mid,awayTeam_def
0,85,83.75,84,80,78.5,79.75
1,78,75.75,75.1667,89,83.5,85
2,86,86,85.6667,75.5,77.2,75.5
3,81,77.2,78,75,73.5,75
4,81,81.5,82.5,78.6667,79.2,77.3333


In [35]:
y.head()

0    HomeWin
1    AwayWin
2    AwayWin
3    AwayWin
4    HomeWin
Name: result, dtype: object

In [36]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

/Users/Michael/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [37]:
# default split is 75% for training and 25% for testing
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1027, 6)
(1027,)
(343, 6)
(343,)


In [139]:
# import model
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth=5, random_state=1)
clf = clf.fit(X_train, y_train)

# instantiate
linreg = LinearRegression()
logreg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
# clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

# fit the model to the training data (learn the coefficients)
# linreg.fit(X_train, y_train)
logreg.fit(X_train, y_train)
# clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [140]:
# print the intercept and coefficients
# print(linreg.intercept_)
# print(linreg.coef_)

# print(logreg.intercept_)
# print(logreg.coef_)

In [141]:
# pair the feature names with the coefficients
# list(zip(feature_cols, logreg.coef_))

In [142]:
# make predictions on the testing set
# y_pred = logreg.predict(X_test)
# y_pred = logreg.predict(X_test)
y_pred = clf.predict(X_test)

In [143]:
# from sklearn import metrics
# print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [144]:
# import seaborn

In [145]:
# seaborn.distplot(y_pred)
# plt.show()

In [146]:
# seaborn.distplot(test_model_df['real_results'])
# plt.show()

In [147]:
# seaborn.distplot(test_model_df['normalized predictions'])
# plt.show()

In [148]:
data = [y_test.values, y_pred]

In [149]:
test_model_df = pd.DataFrame(data, index=['real_results', 'predictions']).transpose()

In [150]:
test_model_df['predictions'].value_counts()

HomeWin    183
AwayWin    155
Draw         5
Name: predictions, dtype: int64

In [151]:
# def normalize_test_model(y):
#     if y > (4/3):
#         return 2
#     elif y < (2/3):
#         return 0
#     else:
#         return 1

In [152]:
counter = 0

for index, row in test_model_df.iterrows():
    if row['real_results'] == row['predictions']:
        counter += 1
    else:
        continue
        
        
counter / len(test_model_df)

0.46355685131195334

In [137]:
# test_model_df['normalized predictions'] = test_model_df['predictions'].apply(normalize_test_model)

In [138]:
# count = 0

# for index, row in test_model_df.iterrows():
#     if row['real_results'] == row['normalized predictions']:
#         count += 1
        
# count / len(test_model_df)